In [119]:
import pandas as pd
import requests

# Load and View Data

In [38]:
wq = pd.read_excel('data/src_uwf_yr1.xlsx', sheet_name=1)
la = pd.read_excel('data/src_uwf_yr1.xlsx', sheet_name=2)
dn = pd.read_excel('data/src_uwf_yr1.xlsx', sheet_name=3)
col = pd.read_excel('data/src_uwf_yr1.xlsx', sheet_name=4)
tss = pd.read_excel('data/src_uwf_yr1.xlsx', sheet_name=5)
ent = pd.read_excel('data/src_uwf_yr1.xlsx', sheet_name=6)
tkn_tp = pd.read_excel('data/src_uwf_yr1.xlsx', sheet_name=7)
lookup = pd.read_excel('data/LookupTable.xls')

In [39]:
lookup

,SRC_ID,UWF_ID,Description,Lat,Long,GlobalID
0,SRC-AH21-23,G9,Oriole Beach (West),30.372390,-87.094826,{3b4c06ae-85d0-4c0c-b25f-e11f5ca811bf}
1,SRC-AH22-23,G1,Oriole Beach (East),30.373026,-87.089196,{9ce48605-b348-4abd-92fc-85fdc49d0207}
2,SRC-AI31-23,H16,Woodlawn Beach,30.387056,-86.991222,{4e2e7ce6-447b-42a0-88f3-623d6d2ab085}
3,SRC-AI42-23,H31,Navarre Beach Wastewater Treatment Plant,30.383276,-86.881788,{a9e9dc36-589f-4aec-b514-e37f9168bdc7}
4,SRC-AI44-23,NB Causeway,Eastern Navigation Channel,30.394471,-86.864446,{38052ba4-ed3b-4f62-b7d7-3a8055392947}
5,SRC-AK41-23,Williams Creek,Williams Creek,30.400667,-86.889756,{947b25d7-b05d-4b2f-bb32-4974002758b1}
6,SRC-AD14-23,NGB4,Western Navigation Channel,30.337960,-87.170290,{5bf91692-3d3d-4c81-81ec-477c9a15e424}
7,SRC-AF24-23,NGB6,Central Navigation Channel #1,30.362540,-87.068840,{8bdf77b9-701d-43a2-a892-0fb94375c3f2}
8,SRC-AI36-23,NGB8,Central Navigation Channel #2,30.386340,-86.945130,{0abb0bae-61bc-4374-bdbc-eff32128bfde}


In [40]:
wq

,Site,Date,Time,Layer,temp (C),percent DO,DO Concentration (mg/L),salinity (PSU),pH,water depth (m),Turbidity (FNU),Secchi (m),weather conditions
0,BSP,2023-08-16,08:50:00,S,29.3,91.7,5.96,26.46,8.01,NaN,0.81,NaN,"camera in shallow, snorkeling photos, 9:35 dee..."
1,EPA,2023-08-16,10:40:00,S,30.3,101.4,6.58,26.66,8.17,NaN,0.45,NaN,shallow camera and 10:50 deep camera
2,G1,2023-01-18,10:09:00,B,15.7,95.8,8.21,24.35,8.06,0.343,0.27,NaN,NaN
3,G1,2023-04-19,10:30:00,B,21.4,117.0,8.92,25.34,8.16,0.990,5.00,NaN,NaN
4,G1,2023-07-19,10:11:00,B,30.9,114.3,NaN,20.55,8.09,NaN,5.10,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,Williams Creek,2023-07-18,09:00:00,B,29.8,32.3,2.09,19.74,7.02,NaN,3.85,NaN,NaN
101,Williams Creek,2022-10-20,06:30:00,S,20.5,63.2,5.00,21.07,7.26,NaN,6.81,NaN,NaN
102,Williams Creek,2023-01-17,09:30:00,S,15.1,61.6,6.13,1.78,6.80,NaN,2.76,NaN,"sunny, partly cloudy (50%0, SW winds about 5 mph"
103,Williams Creek,2023-04-18,09:30:00,S,20.8,76.2,6.43,8.48,6.63,NaN,12.31,0.56,NaN


# Clean and merge data into single dataframe

## Update Site ID's to reflect new naming scheme

In [41]:
# create dictionary with uwf id's as index and src id's as values
repl = lookup.set_index('UWF_ID')['SRC_ID'].to_dict()

#view lookup dictionary
repl

{'G9': 'SRC-AH21-23',
 'G1': 'SRC-AH22-23',
 'H16': 'SRC-AI31-23',
 'H31': 'SRC-AI42-23',
 'NB Causeway': 'SRC-AI44-23',
 'Williams Creek': 'SRC-AK41-23',
 'NGB4': 'SRC-AD14-23',
 'NGB6': 'SRC-AF24-23',
 'NGB8': 'SRC-AI36-23'}

### Water Quality Table

In [42]:
#replace values based on lookup table
wq['Site'].replace(repl, inplace=True)

In [43]:
wq['Site'].value_counts()

Site
SRC-AH22-23          11
SRC-AH21-23          11
NGB 4                10
SRC-AI44-23          10
SRC-AI31-23          10
SRC-AI42-23          10
NGB 8                10
NGB 6                10
SRC-AK41-23           8
Opal Beach (deep)     2
Range Point           2
N9                    1
N7                    1
EPA                   1
N2                    1
N1                    1
NLO                   1
Panferio              1
S10                   1
S16                   1
S18                   1
BSP                   1
Name: count, dtype: int64

In [44]:
#Remove spaces from column values and re-run lookup table replace
wq['Site'] = wq['Site'].apply(lambda x: x.replace(' ', ''))
wq['Site'].replace(repl, inplace=True)

In [45]:
wq['Site'].value_counts()

Site
SRC-AH22-23        11
SRC-AH21-23        11
SRC-AD14-23        10
SRC-AI44-23        10
SRC-AI31-23        10
SRC-AI42-23        10
SRC-AI36-23        10
SRC-AF24-23        10
SRC-AK41-23         8
OpalBeach(deep)     2
RangePoint          2
N9                  1
N7                  1
EPA                 1
N2                  1
N1                  1
NLO                 1
Panferio            1
S10                 1
S16                 1
S18                 1
BSP                 1
Name: count, dtype: int64

### Light Attenuation Table

In [46]:
# Replace values based on lookup table
la['Site'].replace(repl, inplace=True)

In [47]:
la['Site'].value_counts()

Site
SRC-AH21-23    6
SRC-AH22-23    5
SRC-AI31-23    5
SRC-AI42-23    5
SRC-AI44-23    5
NGB 4          5
NGB 6          5
NGB 8          5
BSP            1
EPA            1
NLO            1
Opal Beach     1
Panferio       1
Range Point    1
WC Mouth       1
Name: count, dtype: int64

In [48]:
#Remove spaces from column values and re-run lookup table replace
la['Site'] = la['Site'].apply(lambda x: x.replace(' ', ''))
la['Site'].replace(repl, inplace=True)

In [49]:
la['Site'].value_counts()

Site
SRC-AH21-23    6
SRC-AH22-23    5
SRC-AI31-23    5
SRC-AI42-23    5
SRC-AI44-23    5
SRC-AD14-23    5
SRC-AF24-23    5
SRC-AI36-23    5
BSP            1
EPA            1
NLO            1
OpalBeach      1
Panferio       1
RangePoint     1
WCMouth        1
Name: count, dtype: int64

### Dissolved Nutrients Table

In [50]:
# Replace values based on lookup table
dn['Site'].replace(repl, inplace=True)

In [51]:
dn['Site'].value_counts()

Site
SRC-AH22-23          11
SRC-AH21-23          11
SRC-AF24-23          10
SRC-AI31-23          10
SRC-AI42-23          10
SRC-AI44-23          10
SRC-AI36-23          10
SRC-AK41-23           8
SRC-AD14-23           6
NGB4                  4
Opal Beach            2
Williams Cr Mouth     2
EPA                   1
NLO                   1
Panferio              1
Range Point           1
BSP                   1
Name: count, dtype: int64

In [52]:
# Remove spaces and re-run replace
dn['Site'] = dn['Site'].apply(lambda x: x.replace(' ', ''))
dn['Site'].replace(repl, inplace=True)

In [53]:
dn['Site'].value_counts()

Site
SRC-AH22-23        11
SRC-AH21-23        11
SRC-AI31-23        10
SRC-AI42-23        10
SRC-AI44-23        10
SRC-AD14-23        10
SRC-AF24-23        10
SRC-AI36-23        10
SRC-AK41-23         8
OpalBeach           2
WilliamsCrMouth     2
BSP                 1
EPA                 1
NLO                 1
Panferio            1
RangePoint          1
Name: count, dtype: int64

### Color Table

In [54]:
# Replace values based on lookup table
col['Site'].replace(repl, inplace=True)

In [55]:
col['Site'].value_counts()

Site
SRC-AH22-23    10
SRC-AH21-23    10
SRC-AI31-23    10
SRC-AI42-23    10
SRC-AI44-23    10
SRC-AD14-23    10
SRC-AF24-23    10
SRC-AI36-23    10
SRC-AK41-23     8
Opal Beach      2
WC mouth        2
Name: count, dtype: int64

### Total Suspended Solids

In [56]:
#rename column to site 
tss.rename(columns={'Station' : 'Site'}, inplace=True)
# Replace values based on lookup table
tss['Site'].replace(repl, inplace=True)

In [57]:
#rename Date_Collected Column to Date
tss.rename(columns={'Date_Collected' : 'Date'}, inplace=True)

In [58]:
tss['Site'].value_counts()

Site
SRC-AH21-23    11
SRC-AH22-23     8
SRC-AI31-23     8
SRC-AI42-23     8
SRC-AI44-23     8
SRC-AD14-23     8
SRC-AI36-23     8
SRC-AK41-23     8
SRC-AF24-23     7
BSP             1
EPA             1
NLO             1
Panferio        1
Range Point     1
Name: count, dtype: int64

### Enterococcus table

In [59]:
#rename column to site 
ent.rename(columns={'Station' : 'Site'}, inplace=True)
# Replace values based on lookup table
ent['Site'].replace(repl, inplace=True)

In [60]:
#rename Date_Collected Column to Date
ent.rename(columns={'Date_Collected' : 'Date'}, inplace=True)

In [61]:
ent['Site'].value_counts()

Site
SRC-AH22-23    8
SRC-AH21-23    8
SRC-AI31-23    8
SRC-AI42-23    8
SRC-AI44-23    8
SRC-AD14-23    8
SRC-AF24-23    8
SRC-AI36-23    8
SRC-AK41-23    8
Name: count, dtype: int64

### TKN and TP table

In [62]:
#rename column to site 
tkn_tp.rename(columns={'Station' : 'Site'}, inplace=True)

In [63]:
#rename Date_Collected Column to Date
tkn_tp.rename(columns={'Date_Collected' : 'Date'}, inplace=True)

In [64]:
# Replace values based on lookup table
tkn_tp['Site'].replace(repl, inplace=True)

In [65]:
tkn_tp['Site'].value_counts()

Site
 NB Causeway       8
 NGB8              8
 G9                8
 H31               8
 H16               8
 G1                8
 NGB6              8
 NGB4              8
 Williams Creek    8
Name: count, dtype: int64

In [66]:
# Remove spaces and re-run replace
tkn_tp['Site'] = tkn_tp['Site'].apply(lambda x: x.replace(' ', ''))
tkn_tp['Site'].replace(repl, inplace=True)

In [67]:
tkn_tp['Site'].value_counts()

Site
NBCauseway       8
SRC-AI36-23      8
SRC-AH21-23      8
SRC-AI42-23      8
SRC-AI31-23      8
SRC-AH22-23      8
SRC-AF24-23      8
SRC-AD14-23      8
WilliamsCreek    8
Name: count, dtype: int64

In [68]:
# Manually replace NB Causeway and Williams Creek
tkn_tp['Site'] = tkn_tp['Site'].apply(lambda x: x.replace('WilliamsCreek', 'SRC-AK41-23'))
tkn_tp['Site'] = tkn_tp['Site'].apply(lambda x: x.replace('NBCauseway', 'SRC-AI44-23'))


In [31]:
tkn_tp['Site'].value_counts()

Site
SRC-AI44-23    8
SRC-AI36-23    8
SRC-AH21-23    8
SRC-AI42-23    8
SRC-AI31-23    8
SRC-AH22-23    8
SRC-AF24-23    8
SRC-AD14-23    8
SRC-AK41-23    8
Name: count, dtype: int64

## Prepare Data for Merge

In [227]:
# Add Layer Field to data frames that don't have one. Default layer is Surface
la['Layer'] = 'S'

In [78]:
# Make Sure All 'Layer' values are a single uppercase letter

x = tkn_tp #using 'x' so we only have to change one variable for each of the df's
x['Layer'] = x['Layer'].astype(str).str[0].str.upper()
x['Layer'].value_counts()

Layer
S    36
B    36
Name: count, dtype: int64

In [79]:
# Create Join Field to use as index
wq['Join'] = wq['Site'] + '-' + wq['Layer']
la['Join'] = la['Site'] + '-' + la['Layer']
dn['Join'] = dn['Site'] + '-' + dn['Layer']
col['Join'] = col['Site'] + '-' + col['Layer']
tss['Join'] = tss['Site'] + '-' + tss['Layer']
ent['Join'] = ent['Site'] + '-' + ent['Layer']
tkn_tp['Join'] = tkn_tp['Site'] + '-' + tkn_tp['Layer']

In [80]:
#remove unwanted columns to avoid duplicate data
la.drop(columns = ['Site', 'Layer'], inplace=True, errors = 'ignore')
dn.drop(columns = ['Site', 'Layer'], inplace=True, errors = 'ignore')
col.drop(columns = ['Site', 'Layer'], inplace=True, errors = 'ignore')
tss.drop(columns = ['Site', 'Layer'], inplace=True, errors = 'ignore')
ent.drop(columns = ['Site', 'Layer'], inplace=True, errors = 'ignore')
tkn_tp.drop(columns = ['Site', 'Layer'], inplace=True, errors = 'ignore')

In [81]:
wq['Join'].value_counts()

Join
SRC-AH22-23-S        6
SRC-AH21-23-S        6
SRC-AD14-23-B        5
SRC-AI42-23-S        5
SRC-AI44-23-S        5
SRC-AI44-23-B        5
SRC-AF24-23-S        5
SRC-AI36-23-B        5
SRC-AI36-23-S        5
SRC-AI42-23-B        5
SRC-AD14-23-S        5
SRC-AI31-23-S        5
SRC-AI31-23-B        5
SRC-AH21-23-B        5
SRC-AH22-23-B        5
SRC-AF24-23-B        5
SRC-AK41-23-B        4
SRC-AK41-23-S        4
RangePoint-S         2
EPA-S                1
N9-S                 1
N7-S                 1
N2-S                 1
NLO-S                1
OpalBeach(deep)-B    1
OpalBeach(deep)-S    1
Panferio-S           1
S10-S                1
S16-S                1
S18-S                1
N1-S                 1
BSP-S                1
Name: count, dtype: int64

In [82]:
tss

,Date,TSS,Join
0,2022-09-07,26.000,SRC-AH21-23-S
1,2022-09-07,3.000,SRC-AH21-23-B
2,2022-10-20,3.250,SRC-AH22-23-S
3,2022-10-20,3.000,SRC-AH22-23-B
4,2022-10-20,1.500,SRC-AH21-23-S
...,...,...,...
74,2023-08-16,2.875,SRC-AH22-23-S
75,2023-08-16,3.625,SRC-AH21-23-S
76,2023-08-16,1.875,NLO-S
77,2023-08-16,4.250,Panferio-S


## Merge Data Frames Into Single Data Frame

In [83]:
# Create initial data frame base on WQ data, drop weather conditions
df = wq.drop('weather conditions', axis=1).set_index(['Join', 'Date'])

In [84]:
df = df.merge(dn, how='left', on=['Join', 'Date']).merge(la, how='left', on=['Join', 'Date']).merge(col, how='left', on=['Join', 'Date']).merge(tss, how='left', on=['Join', 'Date']).merge(ent, how='left', on=['Join', 'Date']).merge(tkn_tp, how='left', on=['Join', 'Date'])

In [85]:
df

,Join,Date,Site,Time,Layer,temp (C),percent DO,DO Concentration (mg/L),salinity (PSU),pH,...,Unnamed: 9,Kd /m,Color PCU,TSS,Entero,logEntero,Unnamed: 5,Unnamed: 6,TKN_mgNL,TP_mgPL
0,BSP-S,2023-08-16,BSP,08:50:00,S,29.3,91.7,5.96,26.46,8.01,...,NaN,0.70,NaN,0.625000,NaN,NaN,NaN,NaN,NaN,NaN
1,EPA-S,2023-08-16,EPA,10:40:00,S,30.3,101.4,6.58,26.66,8.17,...,NaN,0.47,NaN,3.125000,NaN,NaN,NaN,NaN,NaN,NaN
2,SRC-AH22-23-B,2023-01-18,SRC-AH22-23,10:09:00,B,15.7,95.8,8.21,24.35,8.06,...,NaN,NaN,8.449159,2.250000,10.0,1.000000,NaN,10.000000,0.243,0.129
3,SRC-AH22-23-B,2023-04-19,SRC-AH22-23,10:30:00,B,21.4,117.0,8.92,25.34,8.16,...,NaN,NaN,5.780856,4.600000,10.0,1.000000,NaN,NaN,0.151,0.137
4,SRC-AH22-23-B,2023-07-19,SRC-AH22-23,10:11:00,B,30.9,114.3,NaN,20.55,8.09,...,NaN,NaN,8.449159,15.555556,10.0,1.000000,NaN,NaN,0.268,0.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,SRC-AK41-23-B,2023-07-18,SRC-AK41-23,09:00:00,B,29.8,32.3,2.09,19.74,7.02,...,NaN,NaN,NaN,68.000000,650.0,2.812913,NaN,NaN,0.604,0.240
102,SRC-AK41-23-S,2022-10-20,SRC-AK41-23,06:30:00,S,20.5,63.2,5.00,21.07,7.26,...,NaN,NaN,NaN,9.000000,189.0,2.276462,NaN,NaN,0.361,0.233
103,SRC-AK41-23-S,2023-01-17,SRC-AK41-23,09:30:00,S,15.1,61.6,6.13,1.78,6.80,...,NaN,NaN,NaN,3.500000,448.0,2.651278,NaN,348.443328,0.408,0.209
104,SRC-AK41-23-S,2023-04-18,SRC-AK41-23,09:30:00,S,20.8,76.2,6.43,8.48,6.63,...,NaN,NaN,NaN,2.750000,279.0,2.445604,NaN,NaN,0.386,0.200


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Join                     106 non-null    object        
 1   Date                     106 non-null    datetime64[ns]
 2   Site                     106 non-null    object        
 3   Time                     106 non-null    object        
 4   Layer                    106 non-null    object        
 5   temp (C)                 106 non-null    float64       
 6   percent DO               106 non-null    float64       
 7   DO Concentration (mg/L)  96 non-null     float64       
 8   salinity (PSU)           106 non-null    float64       
 9   pH                       106 non-null    float64       
 10  water depth (m)          41 non-null     float64       
 11  Turbidity (FNU)          80 non-null     float64       
 12  Secchi (m)               16 non-null

In [87]:
# Drop Remaining Unwanted Columns
df.drop(columns = ['Unnamed: 8','Unnamed: 9','Unnamed: 5','Unnamed: 6', 'logEntero'], inplace = True)

In [88]:
df['Date'].value_counts()

Date
2022-09-07    26
2022-10-20    19
2023-01-18    16
2023-04-19    16
2023-07-19    16
2023-08-16     7
2023-01-17     2
2023-04-18     2
2023-07-18     2
Name: count, dtype: int64

In [89]:
sites = ['SRC-AI44-23',
         'SRC-AI36-23',
         'SRC-AH21-23',
         'SRC-AI42-23',
         'SRC-AI31-23',
         'SRC-AH22-23',
         'SRC-AF24-23',
         'SRC-AD14-23',
         'SRC-AK41-23']

In [90]:
# Remove all sites that are not in the sites list
df = df[df['Site'].isin(sites)]

In [100]:
df.set_index(['Site','Date','Layer']).sort_index(level=['Site','Date'])

Join      Time  temp (C)  percent DO  \
Site        Date       Layer                                                  
SRC-AD14-23 2022-09-07 B      SRC-AD14-23-B  07:20:00      29.7        76.3   
                       S      SRC-AD14-23-S  07:20:00      28.9        97.7   
            2022-10-20 B      SRC-AD14-23-B  08:11:00      20.9        98.7   
                       S      SRC-AD14-23-S  08:11:00      20.0        95.0   
            2023-01-18 B      SRC-AD14-23-B  10:55:00      16.1        94.0   
...                                     ...       ...       ...         ...   
SRC-AK41-23 2023-01-17 S      SRC-AK41-23-S  09:30:00      15.1        61.6   
            2023-04-18 B      SRC-AK41-23-B  09:30:00      23.4        98.0   
                       S      SRC-AK41-23-S  09:30:00      20.8        76.2   
            2023-07-18 B      SRC-AK41-23-B  09:00:00      29.8        32.3   
                       S      SRC-AK41-23-S  09:00:00      26.4        60.5   

                              DO Concentration (mg/L)  salinity (PSU)    pH  \
Site        Date       Layer                                                  
SRC-AD14-23 2022-09-07 B                         4.88           30.02  8.07   
                       S                         6.80           17.44  7.86   
            2022-10-20 B                         6.76           29.63  8.12   
                       S                         7.28           28.85  8.11   
            2023-01-18 B                         7.60           30.24  8.21   
...                                               ...             ...   ...   
SRC-AK41-23 2023-01-17 S                         6.13            1.78  6.80   
            2023-04-18 B                         7.48           20.29  7.67   
                       S                         6.43            8.48  6.63   
            2023-07-18 B                         2.09           19.74  7.02   
                       S                         4.70            2.05  6.35   

                              water depth (m)  Turbidity (FNU) Secchi (m)  \
Site        Date       Layer                                                
SRC-AD14-23 2022-09-07 B                  NaN              NaN        NaN   
                       S                  NaN              NaN        NaN   
            2022-10-20 B                 6.92            -0.49        NaN   
                       S                  NaN            -0.76        NaN   
            2023-01-18 B                 4.50             0.55        NaN   
...                                       ...              ...        ...   
SRC-AK41-23 2023-01-17 S                  NaN             2.76        NaN   
            2023-04-18 B                  NaN          1200.00        NaN   
                       S                  NaN            12.31       0.56   
            2023-07-18 B                  NaN             3.85        NaN   
                       S                  NaN             2.66       0.75   

                              ...  NO3-+NO2- µgN/L  NO2- µgN/L  NH4+ µgN/L  \
Site        Date       Layer  ...                                            
SRC-AD14-23 2022-09-07 B      ...        19.227468    5.905097   19.822614   
                       S      ...         1.802575    0.885764    4.088973   
            2022-10-20 B      ...         4.223020    0.000000    6.394445   
                       S      ...        10.255906    0.000000   10.752211   
            2023-01-18 B      ...         4.837178    1.668874    7.184344   
...                           ...              ...         ...         ...   
SRC-AK41-23 2023-01-17 S      ...              NaN         NaN         NaN   
            2023-04-18 B      ...         4.895706    0.212479   16.931589   
                       S      ...       155.848041    1.345700   28.292152   
            2023-07-18 B      ...         6.698000    0.961606   24.554501   
                       S      ...        27.6

In [95]:
#Sava data to excel file
df.to_excel('data/year_1_clean.xlsx')

In [92]:
df['Layer'].value_counts()

Layer
S    46
B    45
Name: count, dtype: int64

# Prepare data for GIS 

## Add remaining fields to prepare for GIS join 

In [117]:
# Merge remaining fields, drop unwanted.
df = df.merge(lookup, how = 'left', left_on=['Site'], right_on = ['SRC_ID']).drop(columns = ['SRC_ID', 'UWF_ID', 'Description'])

In [126]:
df = df.drop(columns = ['Lat_y', 'Long_y', 'GlobalID_y'])

In [155]:
df = df.rename(columns = {'Lat_x': 'Latitude',
                    'Long_x' : 'Longitude',
                    'GlobalID_x':'ParentGUID'})

## Create weather api call

### Create function for API Call

Function created using chat GPT to help convert javascript to python, then simplified to fit desired needs

In [122]:
def run_weather_calcs(lat, lon, dt):
    # Check to make sure we have latitude, longitude and Datetime
    if lat is None or lon is None or dt is None:
        return ""

    # Format the URL with the input parameters
    key = 'e37258b466e8ec694da96f3d85e99174'
    url = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={dt}&appid={key}&units=imperial"
    
    # Make the request
    response = requests.get(url)

    # Check the response. 200 indicates success from the API
    if response.status_code != 200:
        return f"Failed due to error: {response.status_code} using url: {response.url}"
    else:
        # Check the information in the response for an error
        response_json = response.json()
        if "error" in response_json:
            return response_json["error"]
        else:
            if response_json:
                return response_json
            else:
                return ""       

Now, lets manipulate Date and Time Fields to create Epoch Time for API Call

In [158]:
# Convert Time field to String from Pandas Object

df['Time'] = df['Time'].astype(str)

# Convert 'time' column to timedelta

df['Time'] = pd.to_timedelta(df['Time'])

# Combine 'date' and 'time' into a new column 'datetime'
df['datetime'] = df['Date'] + df['Time']

# Convert 'datetime' column to Unix timestamp (epoch time)
df['epoch_time'] = (df['datetime'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')



In [165]:
df

,Join,Date,Site,Time,Layer,temp (C),percent DO,DO Concentration (mg/L),salinity (PSU),pH,...,Color PCU,TSS,Entero,TKN_mgNL,TP_mgPL,Latitude,Longitude,ParentGUID,datetime,epoch_time
0,SRC-AH22-23-B,2023-01-18,SRC-AH22-23,0 days 10:09:00,B,15.7,95.8,8.21,24.35,8.06,...,8.449159,2.250000,10.0,0.243,0.129,30.373026,-87.089196,{9ce48605-b348-4abd-92fc-85fdc49d0207},2023-01-18 10:09:00,1674036540
1,SRC-AH22-23-B,2023-04-19,SRC-AH22-23,0 days 10:30:00,B,21.4,117.0,8.92,25.34,8.16,...,5.780856,4.600000,10.0,0.151,0.137,30.373026,-87.089196,{9ce48605-b348-4abd-92fc-85fdc49d0207},2023-04-19 10:30:00,1681900200
2,SRC-AH22-23-B,2023-07-19,SRC-AH22-23,0 days 10:11:00,B,30.9,114.3,NaN,20.55,8.09,...,8.449159,15.555556,10.0,0.268,0.100,30.373026,-87.089196,{9ce48605-b348-4abd-92fc-85fdc49d0207},2023-07-19 10:11:00,1689761460
3,SRC-AH22-23-B,2022-09-07,SRC-AH22-23,0 days 13:55:00,B,30.6,103.4,6.92,20.31,8.20,...,5.780856,NaN,NaN,NaN,NaN,30.373026,-87.089196,{9ce48605-b348-4abd-92fc-85fdc49d0207},2022-09-07 13:55:00,1662558900
4,SRC-AH22-23-B,2022-10-20,SRC-AH22-23,0 days 09:05:00,B,18.7,92.8,7.31,28.99,8.19,...,NaN,3.000000,10.0,0.298,0.130,30.373026,-87.089196,{9ce48605-b348-4abd-92fc-85fdc49d0207},2022-10-20 09:05:00,1666256700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,SRC-AK41-23-B,2023-07-18,SRC-AK41-23,0 days 09:00:00,B,29.8,32.3,2.09,19.74,7.02,...,NaN,68.000000,650.0,0.604,0.240,30.400667,-86.889756,{947b25d7-b05d-4b2f-bb32-4974002758b1},2023-07-18 09:00:00,1689670800
87,SRC-AK41-23-S,2022-10-20,SRC-AK41-23,0 days 06:30:00,S,20.5,63.2,5.00,21.07,7.26,...,NaN,9.000000,189.0,0.361,0.233,30.400667,-86.889756,{947b25d7-b05d-4b2f-bb32-4974002758b1},2022-10-20 06:30:00,1666247400
88,SRC-AK41-23-S,2023-01-17,SRC-AK41-23,0 days 09:30:00,S,15.1,61.6,6.13,1.78,6.80,...,NaN,3.500000,448.0,0.408,0.209,30.400667,-86.889756,{947b25d7-b05d-4b2f-bb32-4974002758b1},2023-01-17 09:30:00,1673947800
89,SRC-AK41-23-S,2023-04-18,SRC-AK41-23,0 days 09:30:00,S,20.8,76.2,6.43,8.48,6.63,...,NaN,2.750000,279.0,0.386,0.200,30.400667,-86.889756,{947b25d7-b05d-4b2f-bb32-4974002758b1},2023-04-18 09:30:00,1681810200


First, test API function and save returned values to variable

In [191]:
# Test on single row before interating

latitude = '30.373026' 
longitude = '-87.089196'
timestamp = '1674036540'

weather_data = run_weather_calcs(latitude, longitude, timestamp)

if weather_data:
    # Extract wind and temp values from the API response
    wind_value = weather_data['data'][0]['wind_speed']
    wind_deg = weather_data['data'][0]['wind_deg']
    temp_value = weather_data['data'][0]['temp']
    weather = weather_data['data'][0]['weather'][0]['description']
    
    print(f"Wind: {wind_value}")
    print(f"Wind: {wind_deg}")
    print(f"Temp: {temp_value}")
    print(f"Weather: {weather}")
else:
    print("there was an error")

{'lat': 30.373, 'lon': -87.0892, 'timezone': 'America/Chicago', 'timezone_offset': -21600, 'data': [{'dt': 1674036540, 'sunrise': 1674045905, 'sunset': 1674083516, 'temp': 63.91, 'feels_like': 64.53, 'pressure': 1014, 'humidity': 96, 'dew_point': 62.74, 'clouds': 100, 'visibility': 2816, 'wind_speed': 3.44, 'wind_deg': 80, 'weather': [{'id': 701, 'main': 'Mist', 'description': 'mist', 'icon': '50n'}]}]}
Wind: 3.44
Wind: 80
Temp: 63.91
Weather: mist


Next, use for loop to iterate through dataframe, returning weather data to each record

In [192]:
# Loop through each row of the DataFrame
for index, row in df.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    timestamp = row['epoch_time']

    # Get weather data for the current row
    weather_data = run_weather_calcs(latitude, longitude, timestamp)

    if weather_data:
        # Extract wind and temp values from the API response
        wind_value = weather_data['data'][0]['wind_speed']
        wind_deg = weather_data['data'][0]['wind_deg']
        temp_value = weather_data['data'][0]['temp']
        weather = weather_data['data'][0]['weather'][0]['description']
        
        # Insert values into the DataFrame
        df.at[index, 'Wind'] = wind_value
        df.at[index, 'Wind_Dir'] = wind_deg
        df.at[index, 'Air_Temp'] = temp_value
        df.at[index, 'Weather'] = weather

In [195]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype          
---  ------                   --------------  -----          
 0   Join                     91 non-null     object         
 1   Date                     91 non-null     datetime64[ns] 
 2   Site                     91 non-null     object         
 3   Time                     91 non-null     timedelta64[ns]
 4   Layer                    91 non-null     object         
 5   temp (C)                 91 non-null     float64        
 6   percent DO               91 non-null     float64        
 7   DO Concentration (mg/L)  81 non-null     float64        
 8   salinity (PSU)           91 non-null     float64        
 9   pH                       91 non-null     float64        
 10  water depth (m)          41 non-null     float64        
 11  Turbidity (FNU)          75 non-null     float64        
 12  Secchi (m)              

In [221]:
df_final['Sample_Date'] = df['Date'] + df['Time']
df_final['Weather'] = df_final['Weather'].str.title()
df_final = df.drop(columns = ['datetime', 'epoch_time', 'Join', 'Date', 'Time', 'Longitude', 'Latitude'])

In [216]:
cols = list(df_final.columns)
cols

['Site',
 'Layer',
 'temp (C)',
 'percent DO',
 'DO Concentration (mg/L)',
 'salinity (PSU)',
 'pH',
 'water depth (m)',
 'Turbidity (FNU)',
 'Secchi (m)',
 'chl a µg/L',
 'NO3-+NO2- µgN/L',
 'NO2- µgN/L',
 'NH4+ µgN/L',
 'DIP ugP/L',
 'Kd /m',
 'Color PCU',
 'TSS',
 'Entero',
 'TKN_mgNL',
 'TP_mgPL',
 'ParentGUID',
 'Wind',
 'Wind_Dir',
 'Air_Temp',
 'Weather',
 'Sample_Date']

In [224]:
# Rearrange columns

df_final = df_final[[
    'Sample_Date',
    'Site',
    'Layer',
    'Weather',
    'Wind',
    'Wind_Dir',
    'Air_Temp',
    'temp (C)',
    'percent DO',
    'DO Concentration (mg/L)',
    'salinity (PSU)',
    'pH',
    'water depth (m)',
    'Turbidity (FNU)',
    'Secchi (m)',
    'chl a µg/L',
    'NO3-+NO2- µgN/L',
    'NO2- µgN/L',
    'NH4+ µgN/L',
    'DIP ugP/L',
    'Kd /m',
    'Color PCU',
    'TSS',
    'Entero',
    'TKN_mgNL',
    'TP_mgPL',
    'ParentGUID'  
    ]]

In [225]:
# Set Weather Condition strings to Title Case
df_final['Weather'] = df_final['Weather'].str.title()

In [231]:
pd.set_option('display.max_rows', None)
df_final.set_index(['Site','Sample_Date','Layer']).sort_index(level=['Site','Sample_Date'])

Weather   Wind  Wind_Dir  \
Site        Sample_Date         Layer                                      
SRC-AD14-23 2022-09-07 07:20:00 B             Clear Sky   3.44     320.0   
                                S             Clear Sky   3.44     320.0   
            2022-10-20 08:11:00 B             Clear Sky   4.61      10.0   
                                S             Clear Sky   4.61      10.0   
            2023-01-18 10:55:00 B                  Mist   3.44      90.0   
                                S                  Mist   3.44      90.0   
            2023-04-19 11:13:00 B             Clear Sky   1.01      68.0   
                                S             Clear Sky   1.01      68.0   
            2023-07-19 10:54:00 B            Few Clouds   5.75     230.0   
                                S            Few Clouds   5.75     230.0   
SRC-AF24-23 2022-09-07 13:33:00 B             Clear Sky   6.91      30.0   
                                S             Clear Sky   6.91      30.0   
            2022-10-20 09:30:00 B             Clear Sky   0.00       0.0   
                                S             Clear Sky   0.00       0.0   
            2023-01-18 09:52:00 B                  Mist   3.44      80.0   
                                S                  Mist   3.44      80.0   
            2023-04-19 10:10:00 B             Clear Sky   5.75      30.0   
                                S             Clear Sky   5.75      30.0   
            2023-07-19 09:52:00 B             Clear Sky   4.61     240.0   
                                S             Clear Sky   4.61     240.0   
SRC-AH21-23 2022-09-07 14:11:00 B             Clear Sky   6.91      30.0   
                                S             Clear Sky   6.91      30.0   
            2022-10-20 08:45:00 B             Clear Sky   0.00       0.0   
                                S             Clear Sky   0.00       0.0   
            2023-01-18 10:21:00 B                  Mist   5.82     110.0   
                                S                  Mist   5.82     110.0   
            2023-04-19 10:43:00 B             Clear Sky   0.00       0.0   
                                S             Clear Sky   0.00       0.0   
            2023-07-19 10:25:00 B             Clear Sky   5.75     230.0   
                                S             Clear Sky   5.75     230.0   
            2023-08-16 08:10:00 S             Clear Sky   5.75     350.0   
SRC-AH22-23 2022-09-07 13:55:00 B             Clear Sky   6.91      30.0   
                                S             Clear Sky   6.91      30.0   
            2022-10-20 09:05:00 B             Clear Sky   0.00       0.0   
                                S             Clear Sky   0.00       0.0   
            2023-01-18 10:09:00 B                  Mist   3.44      80.0   
                                S                  Mist   3.44      80.0   
            2023-04-19 10:30:00 B             Clear Sky   0.00       0.0   
                                S             Clear Sky   0.00       0.0   
            2023-07-19 10:11:00 B             Clear Sky   4.61     240.0   
                                S             Clear Sky   4.61     240.0   
            2023-08-16 08:23:00 S             Clear Sky   5.75     350.0   
SRC-AI31-23 2022-09-07 12:55:00 B       Overcast Clouds   9.44       2.0   
                                S       Overcast Clouds   9.44       2.0   
            2022-10-20 10:00:00 B             Clear Sky   9.06      23.0   
                                S             Clear Sky   9.06      23.0   
            2023-01-18 09:29:00 B         Broken Clouds   5.21     164.0   
                                S         Broken Clouds   5.21     164.0   
            2023-04-19 09:45:00 B            Few Clouds   8.97      24.0   
                                S            Few Clouds   8.97      24.0   
            2023-07-19 09:25:00 B             Clear Sky   4.85     215.0   
                    

In [232]:
# Save to excel file

df_final.to_excel('data/WaterQualityTable.xlsx')